<a href="https://colab.research.google.com/github/jaynetra/AIForHealthCare_Mimic3/blob/main/ClinicalQA_SyntheticData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install required packages**

**Langchain, Chromadb, Sentence Embeddings**


In [1]:
pip install langchain

In [2]:
pip install huggingface_hub

In [3]:
pip install transformers

In [4]:

pip install -U sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [5]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 6.5 MB/s eta 0:

In [6]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [7]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

In [8]:
# Get synthetic clinical notes from huggingface
from huggingface_hub import hf_hub_download
import pandas as pd

REPO_ID = "starmpcc/Asclepius-Synthetic-Clinical-Notes"
FILENAME = "synthetic.csv"

dataset = pd.read_csv(
    hf_hub_download(repo_id=REPO_ID, filename=FILENAME, repo_type="dataset")
)

synthetic.csv:   0%|          | 0.00/402M [00:00<?, ?B/s]

In [9]:
dataset.head()

,patient_id,note,question,answer,task
0,0,Discharge Summary:\n\nPatient: 60-year-old mal...,Can you provide a simplified paraphrase of the...,The healthcare team used a gradual approach to...,Paraphrasing
1,1,Discharge Summary:\n\nAdmission Date: [Insert ...,Which coreferences were resolved in the hospit...,The hospital course section resolved the coref...,Coreference Resolution
2,2,Hospital Course Summary:\n\nAdmission Date: [I...,What were the key improvements in the patient'...,"During the hospital course, the patient's medi...",Summarization
3,3,Discharge Summary:\n\nPatient: 69-year-old mal...,What roles did physical therapists have in the...,Physical therapists were responsible for ensur...,Relation Extraction
4,4,Discharge Summary:\n\nPatient Information:\n- ...,What manual airway clearance techniques were u...,The discharge summary stated that 1-2 physical...,Relation Extraction


In [10]:
from langchain_community.document_loaders import DataFrameLoader

In [11]:
loader = DataFrameLoader(dataset[:2], page_content_column="note")

In [12]:
docs = loader.load()
docs

[Document(metadata={'patient_id': 0, 'question': "Can you provide a simplified paraphrase of the sentence, 'To avoid rapid deterioration and respiratory failure, a step-by-step approach was used for position changes' in the patient's discharge summary?", 'answer': "The healthcare team used a gradual approach to changing the patient's position to avoid worsening of the respiratory status and prevent respiratory failure.", 'task': 'Paraphrasing'}, page_content="Discharge Summary:\n\nPatient: 60-year-old male with moderate ARDS from COVID-19\n\nHospital Course:\n\nThe patient was admitted to the hospital with symptoms of fever, dry cough, and dyspnea. During physical therapy on the acute ward, the patient experienced coughing attacks that induced oxygen desaturation and dyspnea with any change of position or deep breathing. To avoid rapid deterioration and respiratory failure, a step-by-step approach was used for position changes. The breathing exercises were adapted to avoid prolonged co

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

In [14]:
texts

[Document(metadata={'patient_id': 0, 'question': "Can you provide a simplified paraphrase of the sentence, 'To avoid rapid deterioration and respiratory failure, a step-by-step approach was used for position changes' in the patient's discharge summary?", 'answer': "The healthcare team used a gradual approach to changing the patient's position to avoid worsening of the respiratory status and prevent respiratory failure.", 'task': 'Paraphrasing'}, page_content='Discharge Summary:\n\nPatient: 60-year-old male with moderate ARDS from COVID-19\n\nHospital Course:\n\nThe patient was admitted to the hospital with symptoms of fever, dry cough, and dyspnea. During physical therapy on the acute ward, the patient experienced coughing attacks that induced oxygen desaturation and dyspnea with any change of position or deep breathing. To avoid rapid deterioration and respiratory failure, a step-by-step approach was used for position changes. The breathing exercises were adapted to avoid prolonged co

In [15]:
# create the open-source embedding function
#embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_function = SentenceTransformerEmbeddings(model_name="medicalai/ClinicalBERT")


<ipython-input-15-d59666d61b4b>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="medicalai/ClinicalBERT")
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [16]:
import chromadb
chroma_client = chromadb.Client()

In [17]:

# load it into Chroma and look at embeddings
db = Chroma.from_documents(texts,embedding=embedding_function)
print(db._collection.get(include=['embeddings']))



{'ids': ['b7c5900d-d0c3-4624-b8c5-5f566c0e94c5', '766a8ea8-b1c2-425b-aa31-3987b90c48f6', '8f90a16a-6fae-4a82-9c1f-0759fbe6d187', 'e1816822-928a-4516-bc80-10f7062f4cdb', 'a6e95059-181e-48c0-bf3b-b84dc16486aa'], 'embeddings': array([[-0.28133163, -0.14185134,  0.1861186 , ...,  0.39873666,
        -0.0721926 , -0.37351754],
       [-0.28432491,  0.05069327,  0.31397551, ...,  0.2726222 ,
        -0.08528646, -0.39258441],
       [-0.31661987, -0.11311773,  0.61498553, ...,  0.56511939,
        -0.18485101, -0.38498446],
       [-0.34581068, -0.15571652,  0.30553469, ...,  0.37923998,
        -0.09221134, -0.35437098],
       [-0.22259755, -0.10641511,  0.2544041 , ...,  0.46623906,
        -0.2375019 , -0.32581058]]), 'documents': None, 'uris': None, 'data': None, 'metadatas': None, 'included': [<IncludeEnum.embeddings: 'embeddings'>]}


In [18]:
# query it
query = "explain the summary"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

Impression: Resolved respiratory tract infection with successful physical therapy for dyspnea relief and anxiety reduction. The patient responded well to therapy and regained his self-confidence, able to walk independently before discharge.


In [19]:
print(docs)

[Document(metadata={'answer': "The hospital course section resolved the coreferences related to the patient's respiratory tract infection by providing information on the initial physical therapy, which focused on dyspnea-relieving positions, mobilization techniques, and deep-breathing exercises, and the modified therapy that targeted the patient's anxiety-induced dyspnea. The patient positively responded to the therapy, alleviating his dyspnea and anxiety, and was eventually able to walk independently before discharge, indicating that his respiratory tract infection had been resolved.", 'patient_id': 1, 'question': "Which coreferences were resolved in the hospital course section related to the patient's respiratory tract infection?", 'task': 'Coreference Resolution'}, page_content='Impression: Resolved respiratory tract infection with successful physical therapy for dyspnea relief and anxiety reduction. The patient responded well to therapy and regained his self-confidence, able to wal

In [20]:
pip install langchain-huggingface

In [22]:


from langchain_community.chat_models import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

from google.colab import userdata
from langchain_huggingface.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModel,pipeline
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
#model = AutoModel.from_pretrained("meta-llama/Llama-3.2-1B")
#pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000,temperature=0)
#hf = HuggingFacePipeline(pipeline=pipe)


hf = HuggingFacePipeline.from_model_id(
    model_id="meta-llama/Llama-3.2-1B",
    task="text-generation",
    model_kwargs={"temperature": 0, "max_length": 64},
    pipeline_kwargs={"max_new_tokens": 1000}
)



tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Device set to use cuda:0


In [23]:
from langchain_core.prompts import PromptTemplate

template = """Question: {question}
Context: {context}
"""

prompt = PromptTemplate.from_template(template)
chain = prompt | hf




In [24]:
question = "Can you provide a simplified paraphrase of the sentence, 'To avoid rapid What does the medical history involve'"

In [25]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
   hf,
    retriever=db.as_retriever()
)

result = qa_chain.invoke({"query": question})
print(result['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Discharge Summary:

Admission Date: [Insert Date]
Discharge Date: [Insert Date]

Patient Name: [Insert Name]
Age/Sex: 39-year-old Male
Medical Record Number: [Insert Number]

Admission Diagnosis: Unspecified respiratory tract infection
Discharge Diagnosis: Resolved respiratory tract infection

Hospital Course: The patient was hospitalized due to persistent fever and dry cough for 2 weeks, leading to reduced general health condition. On admission, the patient required 4 L/min of oxygen due to rapid shallow breathing at rest and severe breathlessness during minor physical activity. The initial physical therapy focused on educating the patient on dyspnea-relieving positions, mobilization techniques, and deep-breathing exercises. However, with time, the patient's anxiety-induced dyspnea became an issue, leading to the modificati